In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
Employee = pd.read_csv('C:\\Users\\om sai\\Downloads\\employee.csv')

In [4]:
Employee.loc[Employee['Attrition']=='No','Attrition'] = 0
Employee.loc[Employee['Attrition']=='Yes','Attrition'] = 1

In [5]:
df =Employee 
df.nunique()
df = df.drop(['EmployeeCount','EmployeeNumber','Over18','StandardHours'],axis=1)

In [6]:
df_numeric = df.select_dtypes(include=['int64'])
df_catg= df.select_dtypes(include=['object'])

In [7]:
df_catg_dum = pd.get_dummies(df_catg)

In [8]:
df_combine = pd.concat([df_catg_dum,df_numeric],axis=1)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_x = df_combine.drop(['Attrition'],axis=1)
train_y =df_combine['Attrition']

In [11]:
X_train,X_test,y_train,y_test =train_test_split(train_x,train_y,test_size=0.3,random_state=20)

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

In [13]:
from sklearn import metrics

In [14]:
print('Accuracy',metrics.accuracy_score(y_test,y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy 0.8662131519274376
Precision: 0.8235294117647058
Recall: 0.2


In [15]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print('Accuracy for tree',metrics.accuracy_score(y_test,y_pred))
print("Precision for tree:",metrics.precision_score(y_test, y_pred))
print("Recall for tree:",metrics.recall_score(y_test, y_pred))

Accuracy for tree 0.8072562358276644
Precision for tree: 0.39436619718309857
Recall for tree: 0.4


In [25]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [26]:
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

GridSearchCV took 65.95 seconds for 72 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.868 (std: 0.007)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'min_samples_split': 2}

Model with rank: 2
Mean validation score: 0.866 (std: 0.009)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 3, 'min_samples_split': 2}

Model with rank: 2
Mean validation score: 0.866 (std: 0.015)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'min_samples_split': 3}



In [27]:

param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1,11),
              "min_samples_split": sp_randint(2,11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

n_iter_search = 20
random_search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 17.03 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.864 (std: 0.009)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 9, 'min_samples_split': 4}

Model with rank: 2
Mean validation score: 0.862 (std: 0.009)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 4, 'min_samples_split': 9}

Model with rank: 3
Mean validation score: 0.861 (std: 0.008)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 6, 'min_samples_split': 2}



In [28]:
random_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
random_search.best_params_

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 9,
 'min_samples_split': 4}

In [30]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features=10,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [31]:
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 10,
 'min_samples_split': 2}